In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.models import load_model 
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config = config)

In [3]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
# In the tf.keras.layers package, layers are objects. To construct a layer,
# simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(10, input_shape = (None, 5))

In [5]:
# To use a layer, simply call it, now available in tf2 eager execution
layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [6]:
# Layers have many useful methods. For example, you can inspect all variables
# in a layer using `layer.variables` and trainable variables using
# `layer.trainable_variables`. In this case a fully-connected layer
# will have variables for weights and biases.
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.29272318,  0.01607591,  0.1298269 ,  0.4058786 ,  0.4950934 ,
          0.04975456,  0.35541213, -0.37367073,  0.59721667, -0.08519   ],
        [ 0.04032451, -0.17809421, -0.62645954, -0.23636892,  0.2946818 ,
         -0.1575534 ,  0.40742522, -0.04191589, -0.08033204,  0.13167495],
        [-0.10653347, -0.1730859 ,  0.45529145,  0.62700254,  0.12032235,
          0.4811682 , -0.2931017 ,  0.24553162, -0.54210556, -0.45455164],
        [ 0.44512624, -0.40537462,  0.17962772,  0.27599496, -0.1099841 ,
         -0.23696077,  0.24940795, -0.38164553, -0.04899001, -0.5088739 ],
        [-0.4773072 , -0.6145875 , -0.38456434,  0.4901238 ,  0.49118525,
          0.24820721,  0.02865583,  0.26761395, -0.3476504 , -0.25629798]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [7]:
# The variables are also accessible through nice accessors
layer.kernel, layer.bias

(<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.29272318,  0.01607591,  0.1298269 ,  0.4058786 ,  0.4950934 ,
          0.04975456,  0.35541213, -0.37367073,  0.59721667, -0.08519   ],
        [ 0.04032451, -0.17809421, -0.62645954, -0.23636892,  0.2946818 ,
         -0.1575534 ,  0.40742522, -0.04191589, -0.08033204,  0.13167495],
        [-0.10653347, -0.1730859 ,  0.45529145,  0.62700254,  0.12032235,
          0.4811682 , -0.2931017 ,  0.24553162, -0.54210556, -0.45455164],
        [ 0.44512624, -0.40537462,  0.17962772,  0.27599496, -0.1099841 ,
         -0.23696077,  0.24940795, -0.38164553, -0.04899001, -0.5088739 ],
        [-0.4773072 , -0.6145875 , -0.38456434,  0.4901238 ,  0.49118525,
          0.24820721,  0.02865583,  0.26761395, -0.3476504 , -0.25629798]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)